In [201]:
import time as clock
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from IPython.display import clear_output, display
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

In [202]:
# configuration of simulator
# and parametrs of reservouir
perm = np.load('perm_3sigma.npy')
nx0, nx1 = perm.shape
nx2 = 1
perm = np.reshape(perm, (nx0, nx1, nx2))
poro = 0.1 + np.zeros((nx0, nx1, nx2))

dx0 = 1.0 / nx0
dx1 = 1.0 / nx1
dx2 = 1.0 / nx2

pwat = 2.0
poil = 4.0
vr = 0.3
kwat = 1.0
koil = 0.3

pmin = 0.0
pmax = 1.0

niter = 5000

t_final = 3.0
dt = t_final / niter

In [203]:
nx0, nx1

(64, 64)

# Read from file

In [204]:
# sim_200 = np.random.randint(0, 5000, 200)
# sim_500 = np.random.randint(0, 5000, 500)
# sim_1000 = np.random.randint(0, 5000, 1000)
# sim_2000 = np.random.randint(0, 5000, 2000)
# np.save('sim_200.npy', sim_200)
# np.save('sim_500.npy', sim_500)
# np.save('sim_1000.npy', sim_1000)
# np.save('sim_2000.npy', sim_2000)

In [240]:
# sim_data: pres, swat, soil
n_points = 100
sim_r = np.load(f'sim_{n_points}.npy')#np.random.randint(0, 5000, n_points)

# 5000
sim_data = np.loadtxt("data_5k_3sigma/sim_5000.txt", dtype=np.float32)[sim_r]
x_list = np.loadtxt("data_5k_3sigma/x_5000.txt", dtype=np.float32)[sim_r]
y_list = np.loadtxt("data_5k_3sigma/y_5000.txt", dtype=np.float32)[sim_r]
t_list = np.loadtxt("data_5k_3sigma/t_5000.txt", dtype=np.float32)[sim_r]

# Convert to torch tensor

In [241]:
simulation_data = torch.tensor(sim_data).requires_grad_(True)

x = torch.tensor(x_list * dx0).requires_grad_(True)
y = torch.tensor(y_list * dx1).requires_grad_(True)
t = torch.tensor(t_list * dt).requires_grad_(True)

points = torch.stack((t, x, y), -1).requires_grad_(True).to('mps')
# points, simulation_data

In [242]:
DEVICE = 'cpu'

In [243]:
class PINN(nn.Module):
    def __init__(self, input_layer = 3, hidden_layer = 64, number_layers = 16, output_layer = 7):
        super().__init__()
        
        activation2 = nn.Sigmoid
        activation1 = nn.ReLU
        
        self.fcs = nn.Sequential(*
            [
                nn.Linear(input_layer, hidden_layer),
                activation1()
            ]
        )
        
        self.fch = nn.Sequential(*
            [
                nn.Sequential(*
                    [
                        nn.Linear(hidden_layer, hidden_layer),
                        activation1()
                    ]
                ) for _ in range(number_layers - 1)
            ]
        )
        
#         resnet = models.resnet18()
        
#         self.resnet_features = nn.Sequential(
#             resnet.layer1,   # First residual block
#             resnet.layer2,   # Second residual block
#             resnet.layer3,   # Third residual block
#             resnet.layer4    # Fourth residual block
#         )
        
#         self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
#         self.fce = nn.Linear(512, output_layer)
        
        self.act = nn.ReLU()
        self.fce = nn.Linear(hidden_layer, output_layer)
        
        
    def forward(self, x):
        """
        function for updating neural network
        
        return:
            x - vector (pres
                        soil
                        swat
                        uoil_x,  
                        uoil_y,
                        uwat_x,  
                        uwat_y)
        """
        
        x = self.fcs(x)
        x = self.fch(x)
        
#         x = x.view(x.size(0), 64, 1, 1)
#         x = self.resnet_features(x)
        
#         x = self.adaptive_pool(x)
        
#         x = x.view(x.size(0), -1)        
        x = self.fce(x)
#         x = self.act(x)
        
        return x

# SVM

In [244]:
def check_current_situation_others(svm_model, scaler, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = svm_model.predict(scaler.transform(test_points))

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()
    
def check_current_situation_xgb(xgb, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = xgb.predict(test_points)

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()

In [245]:
X_train = points.cpu().detach().numpy()
y_train = simulation_data.cpu().detach().numpy()

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

svr = SVR(kernel="rbf", C=1.0, epsilon=0.1, gamma="scale")
model_svr = MultiOutputRegressor(svr)
model_svr.fit(X_train, y_train)
# y_pred = svr.predict(X_test_scaled)
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R^2:", r2_score(y_test, y_pred))

MultiOutputRegressor(estimator=SVR())

In [246]:
# check_current_situation_others(svr, scaler, t=0.0)
# check_current_situation_others(svr, scaler, t=0.01)
# check_current_situation_others(svr, scaler, t=0.02)
# check_current_situation_others(svr, scaler, t=0.03)
# check_current_situation_others(svr, scaler, t=0.04)
# check_current_situation_others(svr, scaler, t=0.05)

# XGBoost

In [247]:
# Example data
xgb_regressor = xgb.XGBRegressor(tree_method='hist', verbosity=2)
xgb_regressor.fit(X_train, y_train)

[01:55:08] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (100, 3, 300).


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [248]:
# check_current_situation_xgb(xgb_regressor, t=0)
# check_current_situation_xgb(xgb_regressor, t=0.01)
# check_current_situation_xgb(xgb_regressor, t=0.02)
# check_current_situation_xgb(xgb_regressor, t=0.03)
# check_current_situation_xgb(xgb_regressor, t=0.04)
# check_current_situation_xgb(xgb_regressor, t=0.05)

# Gaussian Processes Regression

In [249]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) \
         + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gp.fit(X_train, y_train)
# y_pred, sigma = gp.predict(X_test, return_std=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-10. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, normalize_y=True)

In [250]:
# check_current_situation_xgb(gp, t=0)
# check_current_situation_xgb(gp, t=0.01)
# check_current_situation_xgb(gp, t=0.02)
# check_current_situation_xgb(gp, t=0.03)
# check_current_situation_xgb(gp, t=0.04)
# check_current_situation_xgb(gp, t=0.05)

# Metrics Computation

In [251]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    median_absolute_error, 
    mean_absolute_percentage_error
)

def evaluate_multidim_models(models_dict, X_test, y_test, device='cpu'):
    """
    Evaluates models on multi-output regression (Target dim: [N, 3]).
    Output Columns: Pressure, Soil (Oil), Swat (Water).
    
    Parameters:
    - models_dict: Dictionary {'ModelName': model_object}
    - X_test: Test features
    - y_test: Test targets (Must be shape [N, 3])
    - device: 'cpu' or 'cuda' (for PyTorch model)
    
    Returns:
    - pd.DataFrame: Rows are (Variable_Metric), Columns are model names.
    """
    results = {}
    
    # 1. Define the specific output names
    target_names = ["Pressure", "Soil", "Swat"]
    
    # 2. Prepare Ground Truth (y_true) as numpy [N, 3]
    if torch.is_tensor(y_test):
        y_true = y_test.detach().cpu().numpy()
    elif isinstance(y_test, (pd.DataFrame, pd.Series)):
        y_true = y_test.to_numpy()
    else:
        y_true = np.array(y_test)
        
    # Basic shape validation
    if y_true.ndim != 2 or y_true.shape[1] != 3:
        raise ValueError(f"y_test must be shape [N, 3], got {y_true.shape}")

    # 3. Evaluate each model
    for name, model in models_dict.items():
        y_pred = None
        
        # --- Generate Predictions [N, 3] ---
        
        # PyTorch Logic
        if isinstance(model, nn.Module):
            model.eval()
            
            # Prepare input tensor
            if torch.is_tensor(X_test):
                X_tensor = X_test.to(device)
            elif isinstance(X_test, (pd.DataFrame, pd.Series)):
                X_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
            else:
                X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
            
            with torch.no_grad():
                raw_pred = model(X_tensor)[:, [0, 2, 1]]
                y_pred = raw_pred.cpu().numpy() # Keep shape [N, 3]
                
        # Scikit-Learn Logic
        else:
            if torch.is_tensor(X_test):
                X_input = X_test.cpu().numpy()
            else:
                X_input = X_test
                
            y_pred = model.predict(X_input) # Expecting [N, 3] output

        # Ensure prediction shape matches truth
        if y_pred.shape != y_true.shape:
             raise ValueError(f"Model {name} output shape {y_pred.shape} mismatch with y_test {y_true.shape}")

        # --- Calculate Metrics per Column ---
        model_metrics = {}
        
        # Loop over the 3 output dimensions
        for i, label in enumerate(target_names):
            # Extract single column (1D arrays)
            y_t = y_true[:, i]
            y_p = y_pred[:, i]
            
            # Calculate Standard Metrics
            mse = mean_squared_error(y_t, y_p)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_t, y_p)
            r2 = r2_score(y_t, y_p)
            medae = median_absolute_error(y_t, y_p)
            mape = mean_absolute_percentage_error(y_t, y_p)
            
            # Store with prefixed keys (e.g., "Pressure_MSE")
            model_metrics[f"{label}_MSE"] = mse
            model_metrics[f"{label}_RMSE"] = rmse
            model_metrics[f"{label}_MAE"] = mae
            model_metrics[f"{label}_R2"] = r2
            model_metrics[f"{label}_MedAE"] = medae
            model_metrics[f"{label}_MAPE"] = mape

        results[name] = model_metrics

    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

In [252]:
class ModifiedPINN(nn.Module):
    """
    Modified MLP architecture based on Wang et al. with U-Net style connections.
    Particularly effective for capturing sharp gradients in pressure/saturation.
    
    Key improvements:
    - Modified MLP with U and V paths (proven effective for PINNs)
    - Skip connections for gradient flow
    - Separate branches for different physics (pressure vs saturation)
    - Adaptive activation functions
    """
    
    def __init__(self, input_dim=3, hidden_dim=128, num_layers=6, output_dim=7):
        super(ModifiedPINN, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Modified MLP: Two parallel input branches (U and V paths)
        self.U = nn.Linear(input_dim, hidden_dim)
        self.V = nn.Linear(input_dim, hidden_dim)
        
        # Hidden layers with residual connections
        self.hidden_layers = nn.ModuleList()
        for i in range(num_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
        # Learnable scaling parameters for outputs
        self.pressure_scale = nn.Parameter(torch.ones(1))
        self.saturation_scale = nn.Parameter(torch.ones(2))
        self.velocity_scale = nn.Parameter(torch.ones(4))
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Xavier initialization for better gradient flow."""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=1.0)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        """
        Forward pass with modified MLP architecture.
        
        Args:
            x: Input [batch, 7] -> (t, x, y, pwat, poil, kwat, koil)
        
        Returns:
            out: [batch, 7] -> (pres, soil, swat, uoil_x, uoil_y, uwat_x, uwat_y)
        """
        # Modified MLP: Two parallel branches
        U = torch.tanh(self.U(x))
        V = torch.tanh(self.V(x))
        
        # Element-wise multiplication (key feature of modified MLP)
        h = U * V
        
        # Pass through hidden layers with residual connections
        for i, layer in enumerate(self.hidden_layers):
            h_new = torch.relu(layer(h))
            # Residual connection every 2 layers
            if i % 2 == 1 and i > 0:
                h = h + h_new  # Skip connection
            else:
                h = h_new
        
        # Output layer
        out = self.output_layer(h)
        
        # Split outputs
        pressure = out[:, 0:1]
        saturation_oil = out[:, 1:2]
        saturation_wat = out[:, 2:3]
        velocities = out[:, 3:7]
        
        # Apply constraints with learnable scales
        # Pressure: softplus for positivity
        pressure = F.softplus(pressure * self.pressure_scale, beta=1, threshold=20)
        
        # Saturations: sigmoid to enforce [0, 1] bounds
        saturation_oil = torch.sigmoid(saturation_oil * self.saturation_scale[0])
        saturation_wat = torch.sigmoid(saturation_wat * self.saturation_scale[1])
        
        # Enforce saturation constraint: S_o + S_w = 1
        # Normalize to ensure sum equals 1
        sat_sum = saturation_oil + saturation_wat + 1e-8
        saturation_oil = saturation_oil / sat_sum
        saturation_wat = saturation_wat / sat_sum
        
        # Velocities: can be positive or negative, use tanh scaled
        velocities = torch.tanh(velocities) * self.velocity_scale
        
        return torch.cat([pressure, saturation_oil, saturation_wat, velocities], dim=1)

In [253]:
N = 100
pinn = PINN()
# 6 - good, 7 - good
# pinn = ModifiedPINN(
#    input_dim=3,
#    hidden_dim=32,  # Try 128, 256
#    num_layers=4,    # Try 5-8
#    output_dim=7
# ).to(DEVICE)

5
# pinn = ModifiedPINN(
#    input_dim=3,
#    hidden_dim=32,  # Try 128, 256
#    num_layers=4,    # Try 5-8
#    output_dim=7
# ).to(DEVICE)

checkpoint = torch.load(f"pinn_model_weights_{500}-3.pth", map_location='cpu')
pinn.load_state_dict(checkpoint)
print(f"pinn_model_weights_{N}.pth")

pinn_model_weights_100.pth


In [254]:
# sim_data: pres, swat, soil
n_points = 2000
sim_r = np.load(f'sim_{n_points}.npy')#np.random.randint(0, 5000, n_points)

# 5000
sim_data = np.loadtxt("data_5k_3sigma/sim_5000.txt", dtype=np.float32)[sim_r]
x_list = np.loadtxt("data_5k_3sigma/x_5000.txt", dtype=np.float32)[sim_r]
y_list = np.loadtxt("data_5k_3sigma/y_5000.txt", dtype=np.float32)[sim_r]
t_list = np.loadtxt("data_5k_3sigma/t_5000.txt", dtype=np.float32)[sim_r]

In [255]:
simulation_data = torch.tensor(sim_data).requires_grad_(True)

x = torch.tensor(x_list * dx0).requires_grad_(True)
y = torch.tensor(y_list * dx1).requires_grad_(True)
t = torch.tensor(t_list * dt).requires_grad_(True)

points = torch.stack((t, x, y), -1).requires_grad_(True).to('mps')
# points, simulation_data

In [256]:
X_test = points.cpu().detach().numpy()
y_test = simulation_data.cpu().detach().numpy()[:, [0, 1, 2]]

In [257]:
my_models = {
    'PINN': pinn,
    'XGB': xgb_regressor,
    'SVM': model_svr,
    'GP': gp
}

df_metrics = evaluate_multidim_models(my_models, X_test, y_test)
df_metrics

,PINN,XGB,SVM,GP
Pressure_MSE,1.324760e-03,1.285599e-02,1.254677e-02,1.690683e-02
Pressure_RMSE,3.639725e-02,1.133843e-01,1.120124e-01,1.300263e-01
Pressure_MAE,2.393877e-02,7.286885e-02,8.904595e-02,8.830051e-02
Pressure_R2,9.830928e-01,8.359264e-01,8.398728e-01,7.842279e-01
Pressure_MedAE,1.435009e-02,3.946058e-02,7.552066e-02,5.700023e-02
Pressure_MAPE,1.591299e-01,6.225950e-01,9.288205e-01,1.638775e+00
Soil_MSE,9.382806e-04,9.727417e-03,9.756684e-03,1.105443e-02
Soil_RMSE,3.063137e-02,9.862767e-02,9.877593e-02,1.051401e-01
Soil_MAE,1.405716e-02,5.804585e-02,7.684451e-02,6.624591e-02
Soil_R2,9.567381e-01,5.514922e-01,5.501427e-01,4.903066e-01


In [200]:
import matplotlib.pyplot as plt
from pandas.plotting import table

def save_df_as_png(df, filename="metrics.png"):
    # Create a figure. Adjust figsize width/height as needed
    fig, ax = plt.subplots(figsize=(10, 4)) 
    
    # Hide the axes (we only want the table)
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create the table
    # loc='center' centers it
    tab = table(ax, df, loc='center', cellLoc='center', rowLoc='center')

    # Style: Increase font size and scale
    tab.auto_set_font_size(False)
    tab.set_fontsize(12)
    tab.scale(1.2, 1.2) # Scale width and height

    # Save
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

# Usage with your metrics dataframe
# It's usually better to round numbers before plotting with matplotlib

In [25]:
save_df_as_png(df_metrics, filename='metrics_500.png')

In [23]:
df_metrics.values,

(array([[ 0.01553177,  0.01861234,  0.16224783,  0.00163693],
        [ 0.12462652,  0.13642706,  0.40279998,  0.04045901],
        [ 0.08724123,  0.05774736,  0.32621013,  0.02069011],
        [ 0.89996213,  0.88012058, -0.04501502,  0.98945676],
        [ 0.05388868,  0.01841325,  0.27887026,  0.01128623],
        [ 0.04579656,  0.05363928,  0.20882493,  0.01316636]]),)